# Spanner Database Integration Test Notebook

This notebook demonstrates how to connect to your Spanner database, authenticate customers, and retrieve account balances using the code we've created.

## Setup

First, let's install the required packages and set up our environment.

In [1]:
# Install required packages
!pip install google-cloud-spanner


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Create Spanner Connector Module



In [2]:
from google.cloud import spanner
import logging
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Spanner configuration
PROJECT_ID = "resolute-winter-447814-t5"
INSTANCE_ID = "bank-customer-instance"
DATABASE_ID = "bank-customer-db"  

def initialize_spanner_client():
    """
    Initialize and return a Spanner client instance.
    
    Returns:
        A Spanner client instance.
    """
    try:
        client = spanner.Client(project=PROJECT_ID)
        logger.info(f"Successfully initialized Spanner client for project {PROJECT_ID}")
        return client
    except Exception as e:
        logger.error(f"Error initializing Spanner client: {str(e)}")
        raise

def get_database():
    """
    Get a reference to the Spanner database.
    
    Returns:
        A Spanner database instance.
    """
    try:
        client = initialize_spanner_client()
        instance = client.instance(INSTANCE_ID)
        database = instance.database(DATABASE_ID)
        logger.info(f"Successfully connected to database {DATABASE_ID}")
        return database
    except Exception as e:
        logger.error(f"Error connecting to database: {str(e)}")
        raise

def authenticate_customer(customer_id, pin):
    """
    Authenticate a customer by verifying their customer ID and PIN.
    
    Args:
        customer_id (str): The customer's ID.
        pin (str): The customer's PIN.
        
    Returns:
        bool: True if authentication is successful, False otherwise.
    """
    try:
        database = get_database()
        
        with database.snapshot() as snapshot:
            query = """
                SELECT customer_id, pin 
                FROM customers 
                WHERE customer_id = @customer_id
            """
            params = {"customer_id": customer_id}
            param_types = {"customer_id": spanner.param_types.STRING}
            
            results = snapshot.execute_sql(
                query, params=params, param_types=param_types
            )
            
            for row in results:
                stored_pin = row[1]
                if stored_pin == pin:
                    logger.info(f"Customer {customer_id} authenticated successfully")
                    return True
            
            logger.warning(f"Authentication failed for customer {customer_id}")
            return False
    except Exception as e:
        logger.error(f"Error during authentication: {str(e)}")
        return False

def get_customer_details(customer_id):
    """
    Get customer details from the database.
    
    Args:
        customer_id (str): The customer's ID.
        
    Returns:
        dict: A dictionary containing customer details, or None if customer not found.
    """
    try:
        database = get_database()
        
        with database.snapshot() as snapshot:
            query = """
                SELECT customer_id, first_name, last_name, email, phone
                FROM customers 
                WHERE customer_id = @customer_id
            """
            params = {"customer_id": customer_id}
            param_types = {"customer_id": spanner.param_types.STRING}
            
            results = snapshot.execute_sql(
                query, params=params, param_types=param_types
            )
            
            for row in results:
                customer = {
                    "customer_id": row[0],
                    "first_name": row[1],
                    "last_name": row[2],
                    "email": row[3],
                    "phone": row[4]
                }
                return customer
            
            logger.warning(f"Customer {customer_id} not found")
            return None
    except Exception as e:
        logger.error(f"Error retrieving customer details: {str(e)}")
        return None

def get_account_balances(customer_id):
    """
    Get account balances for a customer.
    
    Args:
        customer_id (str): The customer's ID.
        
    Returns:
        list: A list of dictionaries containing account information.
    """
    try:
        database = get_database()
        
        with database.snapshot() as snapshot:
            query = """
                SELECT account_id, account_type, balance, currency
                FROM accounts 
                WHERE customer_id = @customer_id
            """
            params = {"customer_id": customer_id}
            param_types = {"customer_id": spanner.param_types.STRING}
            
            results = snapshot.execute_sql(
                query, params=params, param_types=param_types
            )
            
            accounts = []
            for row in results:
                account = {
                    "account_id": row[0],
                    "account_type": row[1],
                    "balance": float(row[2]),
                    "currency": row[3]
                }
                accounts.append(account)
            
            if not accounts:
                logger.warning(f"No accounts found for customer {customer_id}")
            else:
                logger.info(f"Retrieved {len(accounts)} accounts for customer {customer_id}")
            
            return accounts
    except Exception as e:
        logger.error(f"Error retrieving account balances: {str(e)}")
        return []

## Test Database Connection

Let's test the connection to your Spanner database.

In [3]:
try:
    database = get_database()
    print("✅ Successfully connected to Spanner database!")
except Exception as e:
    print(f"❌ Failed to connect to Spanner database: {str(e)}")

2025-04-03 17:26:14,891 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-03 17:26:14,892 - __main__ - INFO - Successfully connected to database bank-customer-db


✅ Successfully connected to Spanner database!


## Insert Test Customer (Optional)

If you don't have test data in your database yet, you can use this cell to insert a test customer and account. You can skip this step if you already have data in your database.

In [5]:
import uuid
from datetime import datetime

def insert_test_customer():
    try:
        database = get_database()
        
        # Generate a unique customer ID
        customer_id = str(uuid.uuid4())
        
        # Insert customer
        with database.batch() as batch:
            batch.insert(
                table="customers",
                columns=("customer_id", "first_name", "last_name","address", "email", "phone", "pin", "created_at"),
                values=[
                    (customer_id, "Test", "User", "new york","test.user@example.com", "1234567890", "1234", spanner.COMMIT_TIMESTAMP),
                ]
            )
        
        # Insert account
        account_id = str(uuid.uuid4())
        with database.batch() as batch:
            batch.insert(
                table="accounts",
                columns=("customer_id", "account_id", "account_type", "balance", "currency", "created_at"),
                values=[
                    (customer_id, account_id, "Checking", 1000.00, "USD", spanner.COMMIT_TIMESTAMP),
                ]
            )
        
        print(f"✅ Successfully inserted test customer with ID: {customer_id}")
        print(f"✅ Successfully inserted test account with ID: {account_id}")
        print(f"✅ Test customer PIN: 1234")
        
        return customer_id
    except Exception as e:
        print(f"❌ Failed to insert test customer: {str(e)}")
        return None

# Uncomment the line below to insert a test customer
test_customer_id = insert_test_customer()

2025-04-03 17:39:50,075 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-03 17:39:50,076 - __main__ - INFO - Successfully connected to database bank-customer-db


✅ Successfully inserted test customer with ID: 403a5138-4602-4d75-b569-866a2b9c6e59
✅ Successfully inserted test account with ID: dccf897a-c732-4fd5-8aa6-3d86181d657d
✅ Test customer PIN: 1234


## Test Customer Authentication

Now let's test authenticating a customer with their ID and PIN.

In [8]:
# Replace with a valid customer ID from your database
customer_id = "C001"  # If you created a test customer above, use test_customer_id instead
pin = "1434"  # Replace with the correct PIN

auth_result = authenticate_customer(customer_id, pin)

if auth_result:
    print(f"✅ Customer {customer_id} authenticated successfully!")
else:
    print(f"❌ Authentication failed for customer {customer_id}")

2025-04-02 14:26:38,522 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:26:38,523 - __main__ - INFO - Successfully connected to database bank-customer-db
2025-04-02 14:26:38,774 - __main__ - INFO - Customer C001 authenticated successfully


✅ Customer C001 authenticated successfully!


## Get Customer Details

If authentication was successful, let's retrieve the customer details.

In [9]:
if auth_result:
    customer = get_customer_details(customer_id)
    if customer:
        print("✅ Customer details retrieved successfully!")
        print(f"Name: {customer['first_name']} {customer['last_name']}")
        print(f"Email: {customer['email']}")
        print(f"Phone: {customer['phone']}")
    else:
        print("❌ Failed to retrieve customer details")

2025-04-02 14:26:50,850 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:26:50,851 - __main__ - INFO - Successfully connected to database bank-customer-db


✅ Customer details retrieved successfully!
Name: abby Doe
Email: abby.doe@example.com
Phone: 575-183-4767


## Get Account Balances

Finally, let's retrieve the account balances for the authenticated customer.

In [10]:
if auth_result:
    accounts = get_account_balances(customer_id)
    if accounts:
        print(f"✅ Retrieved {len(accounts)} accounts for customer {customer_id}")
        for i, account in enumerate(accounts, 1):
            print(f"\nAccount {i}:")
            print(f"Account ID: {account['account_id']}")
            print(f"Account Type: {account['account_type']}")
            print(f"Balance: {account['balance']} {account['currency']}")
    else:
        print(f"❌ No accounts found for customer {customer_id}")

2025-04-02 14:26:58,041 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:26:58,043 - __main__ - INFO - Successfully connected to database bank-customer-db
2025-04-02 14:26:58,466 - __main__ - INFO - Retrieved 1 accounts for customer C001


✅ Retrieved 1 accounts for customer C001

Account 1:
Account ID: A001
Account Type: Checking
Balance: 2500.75 USD


## Simulate Agent Builder Webhook

Let's simulate how the Agent Builder webhook would work. This is similar to what happens when your agent calls the Cloud Function.

In [11]:
import json

def simulate_webhook_request(action, params):
    print(f"\n=== Simulating webhook request: {action} ===")
    print(f"Request parameters: {json.dumps(params, indent=2)}")
    
    if action == "authenticate":
        customer_id = params.get("customer_id")
        pin = params.get("pin")
        
        if not customer_id or not pin:
            return {"error": "Missing customer_id or pin"}
        
        auth_result = authenticate_customer(customer_id, pin)
        
        if auth_result:
            customer = get_customer_details(customer_id)
            return {
                "authenticated": True,
                "customer": customer
            }
        else:
            return {
                "authenticated": False,
                "message": "Invalid customer ID or PIN"
            }
    
    elif action == "get_balance":
        customer_id = params.get("customer_id")
        
        if not customer_id:
            return {"error": "Missing customer_id"}
        
        accounts = get_account_balances(customer_id)
        
        return {
            "accounts": accounts
        }
    
    else:
        return {"error": f"Unknown action: {action}"}

# Simulate authentication request
auth_response = simulate_webhook_request("authenticate", {
    "customer_id": customer_id,
    "pin": pin
})
print(f"\nResponse: {json.dumps(auth_response, indent=2)}")

# If authentication was successful, simulate balance request
if auth_response.get("authenticated", False):
    balance_response = simulate_webhook_request("get_balance", {
        "customer_id": customer_id
    })
    print(f"\nResponse: {json.dumps(balance_response, indent=2)}")

2025-04-02 14:27:10,042 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:27:10,043 - __main__ - INFO - Successfully connected to database bank-customer-db



=== Simulating webhook request: authenticate ===
Request parameters: {
  "customer_id": "C001",
  "pin": "1434"
}


2025-04-02 14:27:10,380 - __main__ - INFO - Customer C001 authenticated successfully
2025-04-02 14:27:10,385 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:27:10,386 - __main__ - INFO - Successfully connected to database bank-customer-db
2025-04-02 14:27:10,730 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:27:10,731 - __main__ - INFO - Successfully connected to database bank-customer-db



Response: {
  "authenticated": true,
  "customer": {
    "customer_id": "C001",
    "first_name": "abby",
    "last_name": "Doe",
    "email": "abby.doe@example.com",
    "phone": "575-183-4767"
  }
}

=== Simulating webhook request: get_balance ===
Request parameters: {
  "customer_id": "C001"
}


2025-04-02 14:27:11,072 - __main__ - INFO - Retrieved 1 accounts for customer C001



Response: {
  "accounts": [
    {
      "account_id": "A001",
      "account_type": "Checking",
      "balance": 2500.75,
      "currency": "USD"
    }
  ]
}


## Complete Agent Flow Simulation

This cell simulates the complete flow that would happen in my Agent Builder agent.

In [14]:
def simulate_agent_conversation(customer_id, pin):
    print("=== Simulating Agent Builder Conversation ===")
    print("Customer: I need to check my account balance.")
    print("Agent: I'd be happy to help you check your account balance. Could you please provide your customer ID?")
    print(f"Customer: {customer_id}")
    print("Agent: Thank you. For security purposes, could you also provide your PIN?")
    print(f"Customer: {pin}")
    print("Agent: Thank you. Let me verify your information...")
    
    # Authenticate customer
    auth_response = simulate_webhook_request("authenticate", {
        "customer_id": customer_id,
        "pin": pin
    })
    
    if not auth_response.get("authenticated", False):
        print("Agent: I'm sorry, but I couldn't verify your information. Please check your customer ID and PIN and try again.")
        return
    
    customer = auth_response.get("customer", {})
    print(f"Agent: Thank you for verifying your identity, {customer.get('first_name')}. Let me fetch your account balance information...")
    
    # Get account balances
    balance_response = simulate_webhook_request("get_balance", {
        "customer_id": customer_id
    })
    
    accounts = balance_response.get("accounts", [])
    
    if not accounts:
        print("Agent: I'm sorry, but I couldn't find any accounts associated with your customer ID.")
        return
    
    print(f"Agent: I found {len(accounts)} account(s) associated with your customer ID. Here are your account details:")
    
    for i, account in enumerate(accounts, 1):
        print(f"Agent: Account {i}: {account['account_type']} - Balance: {account['balance']} {account['currency']}")
    
    print("Agent: Is there anything else you would like to know about your accounts?")
    print("Customer: No, that's all. Thank you!")
    print("Agent: You're welcome! Have a great day!")

# Uncomment the line below to simulate a complete agent conversation
simulate_agent_conversation(customer_id, pin)

2025-04-02 14:28:25,871 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:28:25,872 - __main__ - INFO - Successfully connected to database bank-customer-db


=== Simulating Agent Builder Conversation ===
Customer: I need to check my account balance.
Agent: I'd be happy to help you check your account balance. Could you please provide your customer ID?
Customer: C001
Agent: Thank you. For security purposes, could you also provide your PIN?
Customer: 1434
Agent: Thank you. Let me verify your information...

=== Simulating webhook request: authenticate ===
Request parameters: {
  "customer_id": "C001",
  "pin": "1434"
}


2025-04-02 14:28:26,222 - __main__ - INFO - Customer C001 authenticated successfully
2025-04-02 14:28:26,226 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:28:26,227 - __main__ - INFO - Successfully connected to database bank-customer-db
2025-04-02 14:28:26,578 - __main__ - INFO - Successfully initialized Spanner client for project resolute-winter-447814-t5
2025-04-02 14:28:26,579 - __main__ - INFO - Successfully connected to database bank-customer-db


Agent: Thank you for verifying your identity, abby. Let me fetch your account balance information...

=== Simulating webhook request: get_balance ===
Request parameters: {
  "customer_id": "C001"
}


2025-04-02 14:28:26,915 - __main__ - INFO - Retrieved 1 accounts for customer C001


Agent: I found 1 account(s) associated with your customer ID. Here are your account details:
Agent: Account 1: Checking - Balance: 2500.75 USD
Agent: Is there anything else you would like to know about your accounts?
Customer: No, that's all. Thank you!
Agent: You're welcome! Have a great day!


## Conclusion

This notebook demonstrates how to connect to your Spanner database, authenticate customers, and retrieve account balances. You can use this as a starting point for testing and developing your Agent Builder integration.

Remember to replace the placeholder values with your actual customer ID and PIN when testing. If you don't have any data in your database yet, you can use the `insert_test_customer()` function to create a test customer and account.

In [15]:
!pip install google-cloud-functions



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [21]:
%%writefile main.py
# Paste the content of your main.py file here
"""
Cloud Function to integrate Spanner database with Agent Builder
This function serves as a webhook for Agent Builder to authenticate customers and retrieve account balances.
"""

import functions_framework
from flask import jsonify
import json
import logging
import os
import sys

# Add the current directory to the path so we can import the spanner_connector module
sys.path.append(os.path.dirname(os.path.realpath(__file__)))
from spanner_connector import authenticate_customer, get_customer_details, get_account_balances

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@functions_framework.http
def bank_agent_webhook(request) :
    """
    HTTP Cloud Function that integrates with Agent Builder.
    This function handles authentication and account balance retrieval requests.
    
    Args:
        request (flask.Request): The request object.
        
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`.
    """
    # Set CORS headers for the preflight request
    if request.method == 'OPTIONS':
        # Allows GET requests from any origin with the Content-Type
        # header and caches preflight response for 3600s
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'POST',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }
        return ('', 204, headers)

    # Set CORS headers for the main request
    headers = {
        'Access-Control-Allow-Origin': '*'
    }
    
    try:
        # Parse the request JSON
        request_json = request.get_json(silent=True)
        if not request_json:
            logger.error("No JSON data in request")
            return jsonify({"error": "No JSON data in request"}), 400, headers
        
        # Log the request (excluding sensitive data)
        logger.info(f"Received request with action: {request_json.get('action', 'unknown')}")
        
        # Get the action from the request
        action = request_json.get('action')
        
        if action == 'authenticate':
            # Handle authentication request
            customer_id = request_json.get('customer_id')
            pin = request_json.get('pin')
            if pin is not None:
                pin = str(pin)
            
            if not customer_id or not pin:
                logger.error("Missing customer_id or pin in authentication request")
                return jsonify({"error": "Missing customer_id or pin"}), 400, headers
            
            # Authenticate the customer
            auth_result = authenticate_customer(customer_id, pin)
            
            if auth_result:
                # Get customer details
                customer = get_customer_details(customer_id)
                return jsonify({
                    "authenticated": True,
                    "customer": customer
                }), 200, headers
            else:
                return jsonify({
                    "authenticated": False,
                    "message": "Invalid customer ID or PIN"
                }), 200, headers
                
        elif action == 'get_balance':
            # Handle balance retrieval request
            customer_id = request_json.get('customer_id')
            
            if not customer_id:
                logger.error("Missing customer_id in balance request")
                return jsonify({"error": "Missing customer_id"}), 400, headers
            
            # Get account balances
            accounts = get_account_balances(customer_id)
            
            return jsonify({
                "accounts": accounts
            }), 200, headers
            
        else:
            logger.error(f"Unknown action: {action}")
            return jsonify({"error": f"Unknown action: {action}"}), 400, headers
            
    except Exception as e:
        logger.error(f"Error processing request: {str(e)}")
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500, headers


Overwriting main.py


In [22]:
%%writefile spanner_connector.py
# Paste the spanner_connector.py code here
"""
Spanner Database Connector for Agent Builder
This module provides functions to connect to a Spanner database and retrieve customer information.
"""

from google.cloud import spanner
import logging
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Spanner configuration
PROJECT_ID = "resolute-winter-447814-t5"
INSTANCE_ID = "bank-customer-instance"
DATABASE_ID = "bank-customer-db"  # Replace with your actual database name

def initialize_spanner_client():
    """
    Initialize and return a Spanner client instance.
    
    Returns:
        A Spanner client instance.
    """
    try:
        client = spanner.Client(project=PROJECT_ID)
        logger.info(f"Successfully initialized Spanner client for project {PROJECT_ID}")
        return client
    except Exception as e:
        logger.error(f"Error initializing Spanner client: {str(e)}")
        raise

def get_database():
    """
    Get a reference to the Spanner database.
    
    Returns:
        A Spanner database instance.
    """
    try:
        client = initialize_spanner_client()
        instance = client.instance(INSTANCE_ID)
        database = instance.database(DATABASE_ID)
        logger.info(f"Successfully connected to database {DATABASE_ID}")
        return database
    except Exception as e:
        logger.error(f"Error connecting to database: {str(e)}")
        raise

def authenticate_customer(customer_id, pin):
    """
    Authenticate a customer by verifying their customer ID and PIN.
    
    Args:
        customer_id (str): The customer's ID.
        pin (str): The customer's PIN.
        
    Returns:
        bool: True if authentication is successful, False otherwise.
    """
    
    try:
        database = get_database()
        
        with database.snapshot() as snapshot:
            query = """
                SELECT customer_id, pin 
                FROM customers 
                WHERE customer_id = @customer_id
            """
            params = {"customer_id": customer_id}
            param_types = {"customer_id": spanner.param_types.STRING}
            
            results = snapshot.execute_sql(
                query, params=params, param_types=param_types
            )
            
            for row in results:
                stored_pin = row[1]
                if stored_pin == pin:
                    logger.info(f"Customer {customer_id} authenticated successfully")
                    return True
            
            logger.warning(f"Authentication failed for customer {customer_id}")
            return False
    except Exception as e:
        logger.error(f"Error during authentication: {str(e)}")
        return False

def get_customer_details(customer_id):
    """
    Get customer details from the database.
    
    Args:
        customer_id (str): The customer's ID.
        
    Returns:
        dict: A dictionary containing customer details, or None if customer not found.
    """
    try:
        database = get_database()
        
        with database.snapshot() as snapshot:
            query = """
                SELECT customer_id, first_name, last_name, email, phone
                FROM customers 
                WHERE customer_id = @customer_id
            """
            params = {"customer_id": customer_id}
            param_types = {"customer_id": spanner.param_types.STRING}
            
            results = snapshot.execute_sql(
                query, params=params, param_types=param_types
            )
            
            for row in results:
                customer = {
                    "customer_id": row[0],
                    "first_name": row[1],
                    "last_name": row[2],
                    "email": row[3],
                    "phone": row[4]
                }
                return customer
            
            logger.warning(f"Customer {customer_id} not found")
            return None
    except Exception as e:
        logger.error(f"Error retrieving customer details: {str(e)}")
        return None

def get_account_balances(customer_id):
    """
    Get account balances for a customer.
    
    Args:
        customer_id (str): The customer's ID.
        
    Returns:
        list: A list of dictionaries containing account information.
    """
    try:
        database = get_database()
        
        with database.snapshot() as snapshot:
            query = """
                SELECT account_id, account_type, balance, currency
                FROM accounts 
                WHERE customer_id = @customer_id
            """
            params = {"customer_id": customer_id}
            param_types = {"customer_id": spanner.param_types.STRING}
            
            results = snapshot.execute_sql(
                query, params=params, param_types=param_types
            )
            
            accounts = []
            for row in results:
                account = {
                    "account_id": row[0],
                    "account_type": row[1],
                    "balance": float(row[2]),
                    "currency": row[3]
                }
                accounts.append(account)
            
            if not accounts:
                logger.warning(f"No accounts found for customer {customer_id}")
            else:
                logger.info(f"Retrieved {len(accounts)} accounts for customer {customer_id}")
            
            return accounts
    except Exception as e:
        logger.error(f"Error retrieving account balances: {str(e)}")
        return []


Overwriting spanner_connector.py


In [23]:
%%writefile requirements.txt
google-cloud-spanner==3.53.0
functions-framework==3.4.0


Overwriting requirements.txt


In [24]:
!gcloud functions deploy bank-agent-webhook \
  --runtime python310 \
  --trigger-http \
  --allow-unauthenticated \
  --entry-point bank_agent_webhook \
  --project resolute-winter-447814-t5 \
  --region us-central1\
  --service-account=232486347340-compute@developer.gserviceaccount.com



Preparing function...done.                                                     
Updating function (may take a while)...                                        
  . [Build]                                                                    
  . [Service]                                                                  
  . [ArtifactRegistry]                                                         
  . [Healthcheck]                                                              
  . [Triggercheck]                                                             
  Updating function (may take a while)...                                      





⠛ Updating function (may take a while)...                                      
  ⠛ [Build]                                                                    




⠹ Updating function (may take a while)...                                      
  ⠹ [Build] Build in progress... Logs are available at [https://console.cloud.g
  oogle.com/cloud-build/builds;